In [73]:
#https://python.langchain.com/v0.1/docs/use_cases/graph/constructing/

! pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j json-repair langsmith

In [74]:
import os
from langsmith import traceable

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "ls__0a4028d0b8844641b9c9416a1b026eb7"


In [75]:

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

#see additional env variables in the docker setup below from LC
#https://python.langchain.com/v0.1/docs/integrations/graphs/neo4j_cypher/

graph = Neo4jGraph()

In [76]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.chat_models.ollama import ChatOllama


#llm = ChatOllama(model="smangrul/llama-3-8b-instruct-function-calling")
llm = ChatOllama(model="phi3")

llm_transformer = LLMGraphTransformer(llm=llm)


In [77]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.  Joan Bonet is the first woman to win a gold medal in shotput.  She is best friends with Pierre Curie.
"""
#Joan Bonet is the first woman to win a gold medal in shotput.  She is best friends with Pierre Curie.
print("Loading document...")
documents = [Document(page_content=text)]
print("Document sucessfully loaded")

print("Converting document to neo4j graph...")
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print("Neo4j schema successfully created ")

print("Loading graph into neo4j")
graph.add_graph_documents(graph_documents)
print("Graph successfully loaded")
#print(f"Nodes:{graph_documents[0].nodes}")
#print(f"Relationships:{graph_documents[0].relationships}")




Loading document...
Document sucessfully loaded
Converting document to neo4j graph...
Neo4j schema successfully created 
Loading graph into neo4j
Graph successfully loaded


In [59]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='1867', type='Year'), Node(id='five Nobel Prizes', type='Number of Awards'), Node(id='Curie family', type='Family'), Node(id='Nobel Prize', type='Award'), Node(id='Marie Curie', type='Person'), Node(id='Pierre Curie', type='Person')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='1867', type='Year'), type='BORN_IN'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Nobel Prize', type='Award'), type='WINNER_OF'), Relationship(source=Node(id='Pierre Curie', type='Person'), target=Node(id='Marie Curie', type='Person'), type='HUSBAND_OF'), Relationship(source=Node(id='Curie family', type='Family'), target=Node(id='five Nobel Prizes', type='Number of Awards'), type='NOBEL PRIZE_WINNERS')]


In [69]:

from langchain_community.document_loaders import TextLoader

loader = TextLoader("/home/jonot480/Documents/jon craig.txt")
doc_text = loader.load()
text=doc_text[0].page_content
documents = [Document(page_content=text)]#there may be an easier way to feed this, but i'm just using the langchain docs 
print(documents)



[Document(page_content='hi my name is jon smith i live in new york.  i am a programmer who is working on ai.  i work with Alan smith.  he is also a programmer.  bill sadd is leading this project.\n')]


In [70]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

graph_documents = llm_transformer.convert_to_graph_documents(documents)



KeyError: 'tail'

In [72]:
print(graph_documents)

[GraphDocument(nodes=[Node(id='jon', type='Person'), Node(id='', type='')], relationships=[Relationship(source=Node(id='jon', type='Person'), target=Node(id='', type=''), type='')], source=Document(page_content='hi my name is jon i live in new york \n', metadata={'id': '4472f58a2593ade483ed55018edf9e91'}))]


In [71]:
graph.add_graph_documents(graph_documents)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `apoc.merge.node`: Caused by: java.lang.IllegalArgumentException: The list of label names may not contain any `NULL` or empty `STRING` values. If you wish to merge a `NODE` without a label, pass an empty list instead.}

In [13]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[Node(id='Pierre Curie', type='Person'), Node(id='Polish and French', type='Country'), Node(id='Marie Curie', type='Person')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Polish and French', type='Country'), type='NATIONALITY'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Pierre Curie', type='Person'), type='SPOUSE')]


In [ ]:
#you need an LLM that can run function calling/tools
#open ai supports this, but you need something like nous hermes or llama3 
#phi3 doesn't seem to work 
#https://www.reddit.com/r/LocalLLaMA/comments/1cl6ocx/llm_with_function_calling_only_openai_models_have/

llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
    node_properties=["born_year"],
)

graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_props[0].nodes}")
print(f"Relationships:{graph_documents_props[0].relationships}")

graph.add_graph_documents(graph_documents_props)


